<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/node_postprocessor/SentenceTransformerRerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rerank can speed up an LLM query without sacrificing accuracy (and in fact, probably improving it). It does so by pruning away irrelevant nodes from the context.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham").load_data()

In [ ]:
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
ctx = ServiceContext.from_defaults(embed_model="local")
set_global_service_context(ctx)

In [ ]:
# build index
index = VectorStoreIndex.from_documents(documents=documents)

In [ ]:
from llama_index.postprocessor import FlagEmbeddingReranker

rerank = FlagEmbeddingReranker(model="BAAI/bge-reranker-large", top_n=3)

First, we try with reranking. We time the query to see how long it takes to process the output from the retrieved context.

In [ ]:
from time import time

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[rerank]
)

now = time()
response = query_engine.query(
    "Which grad schools did the author apply for and why?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

In [ ]:
print(response)

In [ ]:
print(response.get_formatted_sources(length=200))

Next, we try without rerank

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=10)


now = time()
response = query_engine.query(
    "Which grad schools did the author apply for and why?",
)

print(f"Elapsed: {round(time() - now, 2)}s")

In [ ]:
print(response)

In [ ]:
print(response.get_formatted_sources(length=200))

As we can see, the query engine with reranking produced a much more concise output in much lower time (6s v.s. 10s). While both responses were essentially correct, the query engine without reranking included a lot of irrelevant information - a phenomenon we could attribute to "pollution of the context window".